# About: ロギングドライバを変更する--syslog
---

`docker-compose.yml`を編集してMoodleコンテナのロギングドライバを変更します。

## 概要

Dockerコンテナのロギングドライバを`syslog` に変更して、ログを他ホストのsyslogに転送するように設定します。

![処理の流れ](images/moodle-051-01.png)

1. `/var/log/httpd/*` などホスト側に出力されるログファイルを rsyslog で転送する設定を行う
1. ホスト環境の `rsyslogd` を再起動して設定変更を反映する
1. ロギングドライバの設定を追加した`docker-compose.yml`を構築環境に配置する
1. コンテナのロギング設定を変更するためにコンテナを起動し直す


### 前提条件

* Moodle実行環境とは別のノードに syslogサーバが構築済であること
* syslogサーバはMoodle構築環境からのログを受け取れるように設定されていること

## 準備

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切であることをチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

## パラメータ設定

### syslogサーバの指定

ログの送信先となる syslog サーバを指定します。

転送先となるsyslogサーバのホスト名（IPアドレス）を指定してください。

In [ ]:
# (例)
# syslog_server = 'syslog.example.org'
# syslog_server = '192.168.200.200'

syslog_server = 

転送先となるsyslogサーバのポート番号を指定してください。

In [ ]:
# (例)
# syslog_port = 514

syslog_port = 514

転送先となるsyslogサーバのプロトコルを指定してください。

In [ ]:
# (例)
# syslog_protocol = 'tcp'
# syslog_protocol = 'udp'

syslog_protocol = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

指定されたパラメータでログを送信できることを確認します。 転送先となるsyslogサーバでログを受け取ることができたか確認してください。

In [ ]:
logger_opts = '-d' if syslog_protocol == 'udp' else '-T'
logger_message = 'test message 001'

!ansible {target_group} -a \
    'logger -n {syslog_server} -P {syslog_port} {logger_opts} "{logger_message}"'

パラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    logging_driver="syslog",
    syslog_server=syslog_server,
    syslog_port=syslog_port,
    syslog_protocol=syslog_protocol,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

### 転送対象とするホスト環境のログを指定する

コンテナで実行しているApache HTTP serverのなどのサービスのログはホスト環境の`/var/log/`に記録されています。これらのログをホスト側で実行しているrsyslogの機能を利用してリモートホストのsyslogに転送する設定を行います。ここではホスト側のrsyslogで転送するログに関するパラメータの指定を行います。

ホスト側に記録されるログとコンテナとの関係を次表に示します。

|コンテナ|パス|
|:---|:---|
|db|/var/log/mysql/error.log|
|moodle|/var/log/httpd/access_log|
|moodle|/var/log/httpd/error_log|
|httpd|/var/log/httpd/access_log|
|httpd|/var/log/httpd/error_log|
|httpd|/var/log/httpd/ssl_access_log|
|httpd|/var/log/httpd/ssl_error_log|
|shibboleth|/var/log/httpd/access_log|
|shibboleth|/var/log/httpd/error_log|
|shibboleth|/var/log/httpd/ssl_access_log|
|shibboleth|/var/log/httpd/ssl_error_log|
|shibboleth|/var/log/shibboleth/shibd.log|
|shibboleth|/var/log/shibboleth/shibd_warn.log|
|shibboleth|/var/log/shibboleth/signature.log|
|shibboleth|/var/log/shibboleth/transaction.log|


これらのログは、ホスト側で実行しているrsyslogの[imfile](https://rsyslog.readthedocs.io/en/latest/configuration/modules/imfile.html)モジュールの機能によりログの転送を行います。転送対象とするログを次のセルで指定してください。[imfile - Input Parameter](https://rsyslog.readthedocs.io/en/latest/configuration/modules/imfile.html#input-parameters)に示されているパラメータの `File` と `Tag`　が必須項目となります。

例えば、
```python
[
    {
        "file": "/var/log/httpd/access_log",
        "tag": "httpd_access_log",
    },
]
```
のように指定すると、以下に示すような rsyslog の設定が生成されます。

```
input(type="imfile" file="/var/log/httpd/access_log" tag="httpd_access_log" ruleset="moodle")
```

生成された設定のうち`ruleset="moodle"`は、転送先を指定する際のルールセット名になります。

In [ ]:
rsyslog_configs = [
    {
        "file": "/var/log/mysql/error.log",
        "tag": "mariadb_error_log",
    },
    {
        "file": "/var/log/httpd/access_log",
        "tag": "httpd_access_log",
    },
    {
        "file": "/var/log/httpd/access_log",
        "tag": "httpd_error_log",
    },
    {
        "file": "/var/log/httpd/ssl_access_log",
        "tag": "httpd_ssl_access_log",
    },
    {
        "file": "/var/log/httpd/ssl_error_log",
        "tag": "httpd_ssl_error_log",
    },
    {
        "file": "/var/log/shibboleth/shibd.log",
        "tag": "shibboleth_shibd_log",
    },
    {
        "file": "/var/log/shibboleth/shibd_warn.log",
        "tag": "shibboleth_shibd_warn_log",
    },
    {
        "file": "/var/log/shibboleth/signature.log",
        "tag": "shibboleth_signature_log",
    },
    {
        "file": "/var/log/shibboleth/transaction.log",
        "tag": "shibboleth_transaction_log",
    },
]

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    rsyslog_configs=rsyslog_configs,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

### コンテナのログ設定に関するパラメータの指定

コンテナのロギング設定に関するオプションを指定します。指定するパラメータは[Syslog logging driver - Options](https://docs.docker.com/config/containers/logging/syslog/#options)に記されているものをキー、バリューの形で指定します。ただし`syslog-address`については前節で指定したパラメータから内部で自動的に生成するのでここでの指定は不要です。また`tag`については後ほど指定するのでここでの指定は不要です。

In [ ]:
# (例)
# logging_options = {
#    "syslog-facility": "local1",
# }

logging_options = {

}

ロギングオプションの`tag`に指定するパラメータを指定します。指定する値はコンテナごとに異なる値を指定します。次のセルではキーに`docker-compose.yml`のサービス名を、バリューに`tag`に設定する値をdict 型の変数として指定を行います。

「021-Moodleコンテナの起動.ipynb」で構築した環境では`db`コンテナ、`moodle`コンテナの2つのコンテナ飲みで構成されています。「071-HTTPS接続に変更する.ipynb」「221-Shibboleth認証の設定.ipynb」を実行しコンテナ構成が変更された場合を考慮して`httpd`コンテナ、`shibboleth`コンテナに対するタグの値もあらかじめ、ここで指定することを推奨します。

In [ ]:
logging_tags = {
    'db': 'db',
    'moodle': 'moodle',
    'httpd': 'httpd',
    'shibboleth': 'shibboleth',
}

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    logging_driver='syslog',
    logging_options=logging_options if 'logging_options' in vars() else {},
    logging_tags=logging_tags if 'logging_tags' in vars() else {},
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

## rsyslogの設定を変更する



### 設定ファイルを配置する

ログの転送に関する記述を行なった設定ファイルを構築環境に配置します。

実際に設定ファイルを配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/rsyslog/moodle.conf.j2 \
    dest=/etc/rsyslog.d/moodle.conf backup=yes'

設定ファイルを構築環境に配置します。

In [ ]:
!ansible {target_group} -b -m template \
    -a 'src=template/rsyslog/moodle.conf.j2 \
    dest=/etc/rsyslog.d/moodle.conf backup=yes'

### 変更を反映する

rsyslogサービスの再起動を行い、変更を反映させます。

rsyslogサービスの再起動を行います。

In [ ]:
!ansible {target_group} -b -a 'systemctl restart rsyslog'

rsyslogサービスの状態を確認します。エラーになっていないことを確認してください。

In [ ]:
!ansible {target_group} -b -a 'systemctl status rsyslog'

## コンテナのロギング設定を変更する

コンテナのログをsyslogで転送するための構成変更を行います。

### docker-compose.yml の配置

ロギングドライバの設定を追加した`docker-compose.yml`を構築環境に配置します。

実際に`docker-compose.yml`を配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

`docker-compose.yml`を構築環境に配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

### コンテナ設定の変更を反映する

コンテナ設定の変更反映をするためにコンテナを再起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

送信先の syslog にログが転送されていることを確認してください。